In [1]:
import torch
from torch import nn
import numpy as np

In [2]:
import sys
sys.path.append("..")
import gaussian
import qumode
from qumode import QumodeCircuit

## Clements architecture

In [3]:
class Clements(QumodeCircuit):
    def __init__(self, n_modes, r=None, theta=None, phi=None, backend='Gaussian'):
        super().__init__(n_modes, backend)
        self.n_modes = n_modes
        self.backend = backend
        self.r = r
        self.theta = theta
        self.phi = phi
        self.cir()
    
    def cir(self):
        if self.n_modes == 1:
            if self.phi != None:
                super().phase_shift(phi=self.phi[0], mode=0)
            else:
                super().phase_shift(mode=0)
                
        else:
            m = 0
            modes = range(self.n_modes)
            for i in modes:
                for k, (m1, m2) in enumerate(zip(modes[:-1], modes[1:])):
                    if (i + k) % 2 != 1:
                        if self.r != None and theta != None:
                            super().beam_split(r=self.r[m], phi=self.theta[m], mode=[m1, m2])
                        else:
                            super().beam_split(mode=[m1, m2])
                        m += 1

            for i in range(self.n_modes-1):
                if self.phi != None:
                    super().phase_shift(phi=self.phi[i], mode=modes[i])
                else:
                    super().phase_shift(mode=modes[i])

In [4]:
n_modes = 4
batch_size = 1

gbs = Clements(4)

gs = qumode.GaussianState(batch_size, n_modes)

new_gs = gbs(gs)

list(gbs.parameters())

Initialize a gaussian system in the vaccum state with 4 modes and batch size 1.


[Parameter containing:
 tensor(-1.8721, dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor(0.6231, dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor(-0.6025, dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor(0.7909, dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor(0.5349, dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor(0.8365, dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor(-0.5325, dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor(0.0540, dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor(0.7075, dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor(0.2858, dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor(0.7437, dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor(0.9088, dtype=torch.float64, requires_grad=True),
 Parameter containing:
 t

## Squeezing layer

In [4]:
class Squeezing_layer(QumodeCircuit):
    def __init__(self, n_modes, r=None, phi=None, backend='Gaussian'):
        super(Squeezing_layer, self).__init__(n_modes, backend)
        self.n_modes = n_modes
        self.backend = backend
        self.r = r
        self.phi = phi
        self.cir()
    
    def cir(self):
        for i in range(self.n_modes):
            if self.r != None and self.phi != None:
                super().squeeze(r=self.r[i], phi=self.phi[i], mode=i)
            else:
                super().squeeze(mode=i)

In [5]:
n_modes = 4
batch_size = 1

sq = Squeezing_layer(4)

gs = qumode.GaussianState(batch_size, n_modes)

new_gs = sq(gs)

list(sq.parameters())

Initialize a gaussian system in the vaccum state with 4 modes and batch size 1.


[Parameter containing:
 tensor(-0.6331, dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor(0.1706, dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor(-0.3062, dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor(0.5767, dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor(1.2721, dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor(0.6858, dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor(0.2750, dtype=torch.float64, requires_grad=True),
 Parameter containing:
 tensor(0.2586, dtype=torch.float64, requires_grad=True)]

## VarGBS

In [5]:
class VarGBS(nn.Module):
    def __init__(self, n_modes, cle_r=None, cle_theta=None, cle_phi=None, s_r=None, s_phi=None, backend='Gaussian'):
        super().__init__()
        self.n_modes = n_modes
        self.backend = backend
        # parameters for Clements
        self.cle_r = cle_r
        self.cle_theta = cle_theta
        self.cle_phi = cle_phi
        # parameters for Squeezing layer
        self.s_r = s_r
        self.s_phi = s_phi
        
        # squeezing layer
        self.squeeze = Squeezing_layer(self.n_modes, self.s_r, self.s_phi, backend='Gaussian')
        # clements layer utilize a unitary matrix
        self.clements = Clements(self.n_modes, self.cle_r, self.cle_theta, self.cle_phi, backend='Gaussian')
        #self.cir = self.squeeze + self.clements
        
    def forward(self, state):
        state = self.squeeze(state)
        state = self.clements(state)
        return state

In [6]:
n_modes = 4
batch_size = 1

state = qumode.GaussianState(batch_size, n_modes)

gbs = VarGBS(n_modes)

state = gbs(state)

print(list(gbs.parameters()))


Initialize a gaussian system in the vaccum state with 4 modes and batch size 1.
[Parameter containing:
tensor(1.4729, dtype=torch.float64, requires_grad=True), Parameter containing:
tensor(0.2618, dtype=torch.float64, requires_grad=True), Parameter containing:
tensor(-0.6788, dtype=torch.float64, requires_grad=True), Parameter containing:
tensor(0.3760, dtype=torch.float64, requires_grad=True), Parameter containing:
tensor(-1.2161, dtype=torch.float64, requires_grad=True), Parameter containing:
tensor(0.6318, dtype=torch.float64, requires_grad=True), Parameter containing:
tensor(1.4104, dtype=torch.float64, requires_grad=True), Parameter containing:
tensor(0.3925, dtype=torch.float64, requires_grad=True), Parameter containing:
tensor(0.2035, dtype=torch.float64, requires_grad=True), Parameter containing:
tensor(0.1785, dtype=torch.float64, requires_grad=True), Parameter containing:
tensor(-0.7888, dtype=torch.float64, requires_grad=True), Parameter containing:
tensor(0.4909, dtype=torc

In [8]:
n_modes = 3
batch_size = 1
learning_rate = 0.01

# initialize a gaussian state
state = qumode.GaussianState(batch_size, n_modes)

# parameters for squeezing layer
s_r = torch.stack([torch.tensor([0.2])]*n_modes)
s_phi = torch.zeros(n_modes)

# gbs network
gbs = VarGBS(n_modes, s_r=s_r, s_phi=s_phi)

#
state = gbs(state)
state.reset(n_modes, batch_size)

# optimizer
optimizer = torch.optim.Adam(gbs.parameters(), lr=0.01)
#print(list(gbs.parameters()))


# training
n_step = 800
for step in range(n_step):
    # forward 
    state = gbs(state)
    loss = qumode.diff_photon_number(state, 1, 2)
    state.reset(n_modes, batch_size)
    #print(loss)
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if step % 50 == 1:
        print(f'Step: {step}, Loss: {loss}')
        #print(list(gbs.parameters())[0:2])

Initialize a gaussian system in the vaccum state with 3 modes and batch size 1.
Step: 1, Loss: tensor([76.5894], dtype=torch.float64, grad_fn=<SubBackward0>)
Step: 51, Loss: tensor([12.9318], dtype=torch.float64, grad_fn=<SubBackward0>)
Step: 101, Loss: tensor([8.3647], dtype=torch.float64, grad_fn=<SubBackward0>)
Step: 151, Loss: tensor([22.8772], dtype=torch.float64, grad_fn=<SubBackward0>)
Step: 201, Loss: tensor([9.9977], dtype=torch.float64, grad_fn=<SubBackward0>)
Step: 251, Loss: tensor([8.9234], dtype=torch.float64, grad_fn=<SubBackward0>)
Step: 301, Loss: tensor([50.4581], dtype=torch.float64, grad_fn=<SubBackward0>)
Step: 351, Loss: tensor([27.8754], dtype=torch.float64, grad_fn=<SubBackward0>)
Step: 401, Loss: tensor([15.1399], dtype=torch.float64, grad_fn=<SubBackward0>)
Step: 451, Loss: tensor([13.8234], dtype=torch.float64, grad_fn=<SubBackward0>)
Step: 501, Loss: tensor([21.6366], dtype=torch.float64, grad_fn=<SubBackward0>)
Step: 551, Loss: tensor([21.7379], dtype=torch